In [1]:
import pandas as pd
import re
import numpy as np

# Load the Excel file
file_path = 'content/FS_Dataset.xlsx'


In [2]:
# prompt: count and print sheets in the file

xls = pd.ExcelFile(file_path)
print(xls.sheet_names)
print(len(xls.sheet_names))


['Tổng hợp', 'Sheet1', 'Sheet8', 'Lý thuyết', 'Phân ngành', 'Giá', 'Tỷ suất cổ tức', 'Chỉ số', 'Chỉ số 2', 'BCTC 1', 'BCTC 2', 'BCTC 3', 'BCTC 4']
13


In [3]:
sheet_name = 'Tổng hợp'
df = pd.read_excel(file_path, sheet_name=sheet_name)

df.replace('nan', np.nan, inplace=True)
rows, cols = df.shape

print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")


Number of rows: 1665
Number of columns: 847


In [4]:
# df.head()

In [5]:
# #Count unique value of cols 0-6
# for col in df.columns[:6]:
#     counts = df[col].value_counts(dropna=False)
#     print(counts)

In [6]:
#Remain cols
cols = df.columns[6:]

groups = {}
trans_groups = {}

pattern = re.compile(r"(.*?)\n(.*?)\n(.*?)(\d{4})\n(.*?)", re.DOTALL)

for col in cols:
    match = pattern.match(col)
    if match:
        attribute, status, quarter, year, current = match.groups()
        if year not in groups:
            groups[year] = []
        groups[year].append(col)
        if year not in trans_groups:
            trans_groups[year] = []
        trans_groups[year].append(attribute)
        
trans_cols = trans_groups['2010']

In [7]:
#Delete year "2024"
groups.pop('2024', None)
trans_groups.pop('2024', None)

print("In total, there are" ,len(trans_groups), "groups")
for idx,col in enumerate(trans_cols,start=0):
  print('-'*10)
  print(f"Attribute: {idx}: {col}")

In total, there are 14 groups
----------
Attribute: 0: I. TÀI SẢN NGẮN HẠN
----------
Attribute: 1: 1. Tiền và tương đương tiền 
----------
Attribute: 2: 4.1. Hàng tồn kho
----------
Attribute: 3: 3. Các khoản phải thu ngắn hạn
----------
Attribute: 4: 3.1. Phải thu ngắn hạn của khách hàng
----------
Attribute: 5: II. TÀI SẢN DÀI HẠN
----------
Attribute: 6: 2. Tài sản cố định
----------
Attribute: 7: 2. Khấu hao TSCĐ và BĐSĐT (GT)
----------
Attribute: 8: 1. Phải thu dài hạn
----------
Attribute: 9: A. TỔNG CỘNG TÀI SẢN
----------
Attribute: 10: I. NỢ PHẢI TRẢ
----------
Attribute: 11: 1. Nợ ngắn hạn
----------
Attribute: 12: 1.1. Phải trả người bán ngắn hạn
----------
Attribute: 13: 2. Nợ dài hạn
----------
Attribute: 14: II. VỐN CHỦ SỞ HỮU
----------
Attribute: 15: 1.12. Lãi chưa phân phối
----------
Attribute: 16: B. TỔNG CỘNG NGUỒN VỐN
----------
Attribute: 17: 1. Doanh thu bán hàng và cung cấp dịch vụ
----------
Attribute: 18: 3. Doanh thu thuần
----------
Attribute: 19: 4. Giá v

In [8]:
trans_df = pd.DataFrame(columns=list(df.columns[1:6]) + trans_cols)
for year, cols in groups.items():
    new_df = df[cols]
    new_df.columns = trans_cols
    concat_df = pd.concat([df[df.columns[1:6]], new_df], axis=1)
    trans_df = pd.concat([trans_df, concat_df], axis=0, ignore_index=True)
    

C:\Users\Jeff\AppData\Local\Temp\ipykernel_23832\1077537951.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trans_df = pd.concat([trans_df, concat_df], axis=0, ignore_index=True)


In [9]:
# trans_df

# Research Methodology

## Variable measurement

### Z-index of Altman (1968)

In [11]:
# X1 = Current assets minus current liabilities divided by total assets
X1 = trans_df.apply(
    lambda row : (row[trans_cols[0]] - row[trans_cols[11]]) / row[trans_cols[9]]
    if pd.notna(row[trans_cols[0]]) and pd.notna(row[trans_cols[11]]) and pd.notna(row[trans_cols[9]])
    and row[trans_cols[9]] != 0
    else np.nan, axis=1
)

# X2 = Retained earnings divided by total assets
X2 = trans_df.apply(
    lambda row : row[trans_cols[15]] / row[trans_cols[9]]
    if pd.notna(row[trans_cols[15]]) and pd.notna(row[trans_cols[9]])
    and row[trans_cols[9]] != 0
    else np.nan, axis=1
)

# X3 = Profit before tax and interests (EBIT) divided by total assets
X3 = trans_df.apply(
    lambda row : row[trans_cols[44]] / row[trans_cols[9]]
    if pd.notna(row[trans_cols[44]]) and pd.notna(row[trans_cols[9]])
    and row[trans_cols[9]] != 0
    else np.nan, axis=1
)

# X4 = Book value of equity divided by total debt
X4 = trans_df.apply(
    lambda row : row[trans_cols[29]] / row[trans_cols[10]]
    if pd.notna(row[trans_cols[29]]) and pd.notna(row[trans_cols[10]])
    and row[trans_cols[10]] != 0
    else np.nan, axis=1
)

# X5 = Revenue divided by total assets
X5 = trans_df.apply(
    lambda row : row[trans_cols[38]]
    if pd.notna(row[trans_cols[38]])
    else np.nan, axis=1
)

In [18]:
Z_1983 = 0.717*X1 + 0.847*X2 + 3.107*X3 + 0.420*X4 + 0.998*X5

In [ ]:
result = Z_1983.apply(
    lambda row: 2 if row > 2.6 else # non-bankrupt firms
                1 if 1.1 <= row <= 2.6 else # gray area
                0 if row < 1.1 else # high risk of bankruptcy
                np.nan
)

In [ ]:
model_1 = pd.DataFrame(trans_df[trans_df.columns[:5]])
model_1 ['label'] = result
model_1.set_index(model_1.columns[:5].tolist(), inplace=True)

In [ ]:
model_1

# Appendices

## Appendix A

In [13]:
# atr = pd.DataFrame()
# # Index group: Solvency

# # Atr X1: Current ratio: Tỷ số thanh khoản hiện thời
# # Measurement: Current assets/Short-term liabilities: Tài sản ngắn hạn / Nợ ngắn hạn
# for year, cols in groups.items():
#     col1, col2 = cols[0], cols[11]
#     result_col = df.apply(
#         lambda row: row[col1] / row[col2] if pd.notna(row[col1]) and pd.notna(row[col2])
#         and row[col2] != 0 else np.nan, axis=1
#     )
#     col_name = f"Current ratio: X1: {year}"
#     atr[col_name] = result_col

# # Atr X2: Quick ratio: Tỷ số thanh toán nhanh
# # Measurement: (Current assets – inventories)/Current liabilities: (Tài sản ngắn hạn – hàng tồn kho) / Nợ ngắn hạn
# for year, cols in groups.items():
#     col1, col2, col3 = cols[0], cols[2], cols[11]
#     result_col = df.apply(
#         lambda row: (row[col1] - row[col2])/row[col3] if pd.notna(row[col1]) and pd.notna(row[col2])
#         and pd.notna(row[col3]) and row[col3] != 0 else np.nan, axis=1
#     )
#     col_name = f"Quick ratio: X2: {year}"
#     atr[col_name] = result_col
    
# # Atr X3: Receivables turnover: Vòng quay khoản phải thu
# # Measurement: Net revenue/Receivables
# for year, cols in groups.items():
#     col1 = cols[35]
#     result_col = df.apply(
#         lambda row: row[col1] if pd.notna(row[col1]) else np.nan, axis=1
#     )
#     col_name = f"Receivables turnover: X3: {year}"
#     atr[col_name] = result_col
    
# # Atr X4: Ratio of operating cash flow to short-term debt
# # Measurement: Operating cash flow/Short-term debt
# for year, cols in groups.items():
#     col1 = cols[35]
#     result_col = df.apply(
#         lambda row: row[col1] if pd.notna(row[col1]) else np.nan, axis=1
#     )
#     col_name = f"Receivables turnover: X3: {year}"
#     atr[col_name] = result_col